# Chronological frequency in OCRE

This notebook looks at the frequency of minting in two different ways:

- number of issues per year
- frequency of issues(measured in issues per annum) by emperor or issuing authority.  

It uses version `3.0.0` of the `nomisma` library.



## Configure Jupyter notebook

First configure the Jupyter notebook. In addition to the `nomisma` library, we will use `plotly` for graph plots, and a `histoutils` package to simplify working with histograms.

In [1]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:3.0.0`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                      

## Load the full OCRE data set

In [3]:
import edu.holycross.shot.nomisma._
val ocreCex = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
val ocre = OcreSource.fromUrl(ocreCex)

// Sanity check:
println(ocre.size + " records loaded.") 


Dec 27, 2019 10:12:20 PM wvlet.log.Logger log
INFO: Reading 50644 lines of CEX data.
2019-12-27 22:12:21.454Z  info [OcreSource] Created Ocre with 50644 issues.  - (OcreSource.scala:33)


50644 records loaded.


import edu.holycross.shot.nomisma._

ocreCex: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
ocre: Ocre = Ocre(
  Vector(
    OcreIssue(
      "3.com.43",
      "RIC III Commodus 43",
      "denarius",
      "ar",
      "commodus",
      "rome",
      "italy",
      "Head of Commodus, laureate, right",
      "M COMMODVS ANTONINVS AVG",
      "http://nomisma.org/id/commodus",
      "Roma, helmeted, draped, standing left, holding Victory in extended right hand and vertical spear in left hand",
      "TR P VII IMP V COS III P P",
      "http://collection.britishmuseum.org/id/person-institution/60208",
      Some(YearRange(182, Some(182)))
    ),
    OcreIssue(
      "9.thes.27B.iii",
      "RIC IX Thessalonica 27B: Subtype iii",
      "ae3",
      "ae",
      "valentinian_i",
      "thessalonica",
      "macedonia",
      "Bust of Valens, pearl-diademed, draped and cuirassed, right",
      "D N VALEN-S P F AVG",
      "http://nomisma.org/id/val

Set up environment for graph plotting with ploty.

In [4]:
// Import plotly libraries, and set display defaults suggested for use in Jupyter NBs:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import edu.holycross.shot.histoutils._



import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import edu.holycross.shot.histoutils._



## Chronological summary

Overview of the whole data set:

In [5]:
// Use the date range object:
println("Total number of issues in OCRE: " + ocre.size)
println("Number of datable issues: " + ocre.datable.size)

println("Chronological range of issues in OCRE: " + ocre.dateRange.toString(" - "))
println("Span of years: " + ocre.dateSpan)


val avgFreq = ocre.size * 1.0f / ocre.dateSpan
println("Average annual rate of striking: " + avgFreq + " issues per year")

Total number of issues in OCRE: 50644
Number of datable issues: 50613
Chronological range of issues in OCRE: 32 BCE - 491 CE
Span of years: 524
Average annual rate of striking: 96.64886 issues per year


avgFreq: Float = 96.64886F

## Visualize annual rates

In [6]:
val groupedByYear = ocre.datable.issues.groupBy(_.dateRange.get.pointAverage)
val yearFreqs = groupedByYear.toVector.map{ case (yr, vect) => Frequency(yr, vect.size)}
val yearHisto = edu.holycross.shot.histoutils.Histogram(yearFreqs)

groupedByYear: Map[Int, Vector[OcreIssue]] = Map(
  69 -> Vector(
...
yearFreqs: Vector[Frequency[Int]] = Vector(
  Frequency(69, 263),
...
yearHisto: edu.holycross.shot.histoutils.Histogram[Int] = Histogram(
  Vector(
...

In [7]:
val years = yearHisto.frequencies.map(_.item)
val yearCounts = yearHisto.frequencies.map(_.count)

val annualPlot = Seq(
  Bar(x = years, y = yearCounts)
)
plot(annualPlot)

years: Vector[Int] = Vector(
  69,
...
yearCounts: Vector[Int] = Vector(
  263,
...
annualPlot: Seq[Bar] = List(
  Bar(
...
res6_3: String = "plot-0d458486-7ee6-423b-b2ac-9e8e8c9a5a68"

## Visualize rate of issues per authority

Start with a chronologically ordered list of `Ocre` objects grouped by issuing authority:

In [8]:
val byAuthorityChronological = ocre.datable.byAuthority
val authNames = byAuthorityChronological.map(_._1)
val authOcres =  byAuthorityChronological.map(_._2)


byAuthorityChronological: Vector[(String, Ocre)] = Vector(
  (
...
authNames: Vector[String] = Vector(
  "augustus",
...
authOcres: Vector[Ocre] = Vector(
  Ocre(
...

List authorities chronologically:

In [9]:
val summaries = for (auth <- authNames) yield {
    print(auth + ": ")
    val ocreForAuth =  Ocre(ocre.issuesForAuthority(auth))
    println(ocreForAuth.size + " issues in " + ocreForAuth.dateSpan + " years.")
    println(" == " + ocreForAuth.size * 1.0f / ocreForAuth.dateSpan)
    (auth, ocreForAuth.size, ocreForAuth.dateSpan,  ocreForAuth.size * 1.0f / ocreForAuth.dateSpan)
}

augustus: 629 issues in 47 years.
 == 13.382978
tiberius: 95 issues in 24 years.
 == 3.9583333
gaius: 63 issues in 5 years.
 == 12.6
claudius: 126 issues in 14 years.
 == 9.0
nero: 622 issues in 15 years.
 == 41.466667
clodius_macer: 42 issues in 1 years.
 == 42.0
galba: 522 issues in 2 years.
 == 261.0
otho: 24 issues in 1 years.
 == 24.0
vitellius: 177 issues in 1 years.
 == 177.0
vespasian: 1588 issues in 11 years.
 == 144.36363
titus: 543 issues in 3 years.
 == 181.0
domitian: 859 issues in 16 years.
 == 53.6875
nerva: 177 issues in 3 years.
 == 59.0
trajan: 942 issues in 20 years.
 == 47.1
hadrian: 2434 issues in 23 years.
 == 105.82609
antoninus_pius: 2381 issues in 24 years.
 == 99.208336
marcus_aurelius: 1819 issues in 20 years.
 == 90.95
anonymous: 184 issues in 208 years.
 == 0.88461536
commodus: 945 issues in 16 years.
 == 59.0625
pertinax: 71 issues in 1 years.
 == 71.0
pescennius_niger: 196 issues in 2 years.
 == 98.0
didius_julianus: 52 issues in 1 years.
 == 52.0
clodius

summaries: Vector[(String, Int, Int, Float)] = Vector(
  ("augustus", 629, 47, 13.382978F),
...

In [10]:
val issueFreqPlot = Seq(
  Bar(x = authNames, y = summaries.map(_._4))
)
plot(issueFreqPlot)

issueFreqPlot: Seq[Bar] = List(
  Bar(
...
res9_1: String = "plot-7ec90a36-95e6-48a6-ba9c-58fc285568aa"

In [11]:
plot(annualPlot)

res10: String = "plot-b2802aa7-c146-4d5e-830e-2fde462e9547"